In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
from pymongo import MongoClient

from src.config_tickets import ticket_lst
from src.scraping import WebScraping
from src.settings import HOST
from src.query_data import QueryData
from src.utilities import run_model_with_parameters

In [3]:
# Display and Plotting
import matplotlib.pylab as plt
import seaborn as sns

from ipywidgets import interactive, widgets, RadioButtons, ToggleButton, Select, FloatSlider, FloatRangeSlider, IntSlider, fixed

pd.set_option('display.float_format', lambda x: '%.5f' % x) # pandas
np.set_printoptions(precision=5, suppress=True) # numpy

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# seaborn plotting style
sns.set(style='ticks', context='poster')

In [4]:
def connect_2_dbServer():
    mongoClient = MongoClient(HOST)
    return mongoClient

In [5]:
client = connect_2_dbServer()
query = QueryData(client)

In [6]:
lst_ticket = query.get_list_ticket(index='VN 30 (VNI30)')
print(lst_ticket)

['Ho Chi Minh City Infrastructure Investment JSC', 'Ho Chi Minh City Development Joint Stock Commercial Bank', 'Hoa Phat Group JSC', 'Masan Group Corp', 'Military Commercial Joint Stock Bank', 'Mobile World Investment Corp', 'No Va Land Investment Group Corp', 'PetroVietnam Fertilizer and Chemicals Corp', 'Phu Nhuan Jewelry JSC', 'Refrigeration Electrical Engineering Corp', 'Sai Gon Thuong Tin Commercial Joint Stock Bank', 'Saigon Beer Alcohol Beverage Corp', 'Saigon Securities Incorporation', 'Vietnam Technological And Commercial Joint Stock Bank', 'Thanh Thanh Cong Tay Ninh JSC', 'Joint Stock Commercial Bank for Foreign Trade of Vietnam', 'Vietjet Aviation JSC', 'Vietnam Export Import Commercial Joint Stock Bank', 'Vietnam JSCmmercial Bank for Industry and Trade', 'Vietnam Prosperity Joint Stock Commercial Bank', 'Vietnam Dairy Products JSC', 'Vincom Retail JSC', 'Vingroup JSC', 'Vinhomes JSC', 'Petrovietnam Gas JSC']


In [7]:
start = datetime(2010,1,1)
end = datetime(2019,1,1)

df = query.get_historical_data(lst_ticket[:5], start=start)
closed_price = dict()
for ticket in lst_ticket[:5]:
    closed_price[ticket] = df[df.name == ticket]['close']

## Plot ACF and PACF

In [8]:
from statsmodels.tsa.stattools import pacf, acf

def calculate_acf(time_series, lag=20, alpha=0.01):
    x = time_series.values
    acf_value, confint = acf(x, nlags=lag, alpha=alpha)
    confint_lower = confint[:, 0] - acf_value
    confint_upper = confint[:, 1] - acf_value
    return acf_value, confint_upper, confint_lower


def calculate_pacf(time_series, lag=20, alpha=0.01):
    x = time_series.values
    pacf_value, confint = pacf(x, nlags=lag, alpha=alpha)
    confint_lower = confint[:, 0] - pacf_value
    confint_upper = confint[:, 1] - pacf_value
    return pacf_value, confint_upper, confint_lower

## Run model

### ARIMA model

In [9]:
for ticket in lst_ticket[:5]:
    time_series = df[df.name == ticket]['close']
    result = run_model_with_parameters(time_series, model_selection='ARIMA')
    time.sleep(1)
    print(ticket)
    print('Training result:', result['train_evaluation'])
    print('Testing result:', result['test_evaluation'])

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Ho Chi Minh City Infrastructure Investment JSC
Training result: {'mse': 1528427.1551039552, 'rmse': 1236.2957393374593, 'mae': 914.2229254063786, 'mape': 3.4072648055471966}
Testing result: {'mse': 469982.5635552166, 'rmse': 685.5527430878069, 'mae': 593.2397727274613, 'mape': 2.487501418118101}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Ho Chi Minh City Development Joint Stock Commercial Bank
Training result: {'mse': 734718.4223398807, 'rmse': 857.1571748167781, 'mae': 719.1308315885542, 'mape': 2.457940773079639}
Testing result: {'mse': 166214.47565128782, 'rmse': 407.6940956787182, 'mae': 373.40884338643326, 'mape': 1.3804961732623877}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Hoa Phat Group JSC
Training result: {'mse': 776865.3481956312, 'rmse': 881.3996529359603, 'mae': 658.9848236834844, 'mape': 1.9249704193309034}
Testing result: {'mse': 719950.6418130983, 'rmse': 848.4990523348263, 'mae': 586.5094891612478, 'mape': 1.7948011621466347}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Masan Group Corp
Training result: {'mse': 13155740.653682549, 'rmse': 3627.084318523978, 'mae': 2193.2723989173296, 'mape': 2.6441500701086804}
Testing result: {'mse': 4237839.185952517, 'rmse': 2058.6012692973154, 'mae': 1424.5722968200914, 'mape': 1.6473309680206045}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Military Commercial Joint Stock Bank
Training result: {'mse': 568053.9197429796, 'rmse': 753.6935184429939, 'mae': 560.6947614956272, 'mape': 2.661135314010344}
Testing result: {'mse': 383805.3350214576, 'rmse': 619.5202458527547, 'mae': 479.70724536429213, 'mape': 2.2737040367882098}


### ANN model

In [10]:
for ticket in lst_ticket[:5]:
    time_series = df[df.name == ticket]['close']
    result = run_model_with_parameters(time_series, model_selection='ANN')
    time.sleep(1)
    print(ticket)
    print('Training result:', result['train_evaluation'])
    print('Testing result:', result['test_evaluation'])

Ho Chi Minh City Infrastructure Investment JSC
Training result: {'mse': 961165.8066176894, 'rmse': 980.3906398052204, 'mae': 714.8648720144886, 'mape': 2.7782934385628493}
Testing result: {'mse': 427087.53178132547, 'rmse': 653.5193430812324, 'mae': 559.4648079444545, 'mape': 2.3380751018065604}
Ho Chi Minh City Development Joint Stock Commercial Bank
Training result: {'mse': 689164.7340562785, 'rmse': 830.1594630288077, 'mae': 706.9567843923963, 'mape': 2.4052389308762474}
Testing result: {'mse': 184993.17037085633, 'rmse': 430.10832399624206, 'mae': 371.84294728200985, 'mape': 1.3722044533962285}
Hoa Phat Group JSC
Training result: {'mse': 1199616.0904186007, 'rmse': 1095.2698710448492, 'mae': 860.8871552277121, 'mape': 2.465691285932503}
Testing result: {'mse': 517387.41704119794, 'rmse': 719.296473674936, 'mae': 552.7155236992571, 'mape': 1.6934660331555824}
Masan Group Corp
Training result: {'mse': 12768197.800845206, 'rmse': 3573.2615074809746, 'mae': 2537.8331296627657, 'mape': 

### Hybrid model

In [11]:
for ticket in lst_ticket[:5]:
    time_series = df[df.name == ticket]['close']
    result = run_model_with_parameters(time_series, model_selection='Hybrid')
    time.sleep(1)
    print(ticket)
    print('Training result:', result['train_evaluation'])
    print('Testing result:', result['test_evaluation'])

/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Ho Chi Minh City Infrastructure Investment JSC
Training result: {'mse': 1083152.0403941283, 'rmse': 1040.7459057782203, 'mae': 781.0790663131805, 'mape': 3.0239406286107924}
Testing result: {'mse': 543039.9367330978, 'rmse': 736.9124349155046, 'mae': 624.2998135276372, 'mape': 2.620166519530568}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Ho Chi Minh City Development Joint Stock Commercial Bank
Training result: {'mse': 607773.1220033577, 'rmse': 779.5980515646237, 'mae': 621.6751677594534, 'mape': 2.11873989972792}
Testing result: {'mse': 428807.2023616961, 'rmse': 654.8337211549938, 'mae': 622.9707447552197, 'mape': 2.3105427143361683}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Hoa Phat Group JSC
Training result: {'mse': 801328.102462145, 'rmse': 895.1693149690426, 'mae': 679.8606412314123, 'mape': 1.9971260542346299}
Testing result: {'mse': 1078281.9203000832, 'rmse': 1038.4035440521586, 'mae': 783.7505439223044, 'mape': 2.3914468578490404}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Masan Group Corp
Training result: {'mse': 13807754.75049805, 'rmse': 3715.8787319418875, 'mae': 2435.0276823690297, 'mape': 2.9339177525582567}
Testing result: {'mse': 7683292.480951961, 'rmse': 2771.8752643205216, 'mae': 2422.004750379123, 'mape': 2.798954980513125}


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Military Commercial Joint Stock Bank
Training result: {'mse': 583932.1646598423, 'rmse': 764.1545423929915, 'mae': 568.0735898691066, 'mape': 2.6889364465077112}
Testing result: {'mse': 445895.53879026667, 'rmse': 667.7541005417089, 'mae': 544.8174085931306, 'mape': 2.5625124198352447}
